In [ ]:
#| default_exp learning_context

# Learning Context

> Core module for managing learning context (memory -> LOs, prior cases, knowledge gaps, feedback preferences)

## Setup

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Optional
from datetime import datetime
from enum import Enum
from pydantic import BaseModel, Field
import json
from pathlib import Path
import logging
from wardbuddy.utils import setup_logger, load_context_safely, save_context_safely

C:\Users\deepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
logger = setup_logger(__name__)

## Learning Context Management

> Core module for managing student learning context and history

The system needs to track and handle student information for personalised education.  

This module handles:
* Tracking learning objectives
* Managing case history
* Monitoring knowledge gaps
* Customizing feedback preferences

and finally:
* Context persistence


In [ ]:
#| export
class LearningCategory(str, Enum):
    """Main learning categories"""
    HISTORY_TAKING = "History Taking"
    PHYSICAL_EXAM = "Physical Examinations"
    INVESTIGATIONS = "Investigations"
    MANAGEMENT = "Management"
    CLINICAL_REASONING = "Clinical Reasoning"
    COMMUNICATION = "Communication & Presentation"
    PROCEDURAL = "Procedural Skills"
    MEDICAL_KNOWLEDGE = "Medical Knowledge"

class RotationContext(BaseModel):
    """Clinical rotation context"""
    specialty: str = Field(..., description="Medical specialty")
    setting: str = Field(..., description="Clinical setting (Clinic/Wards/ED)")

class SmartGoal(BaseModel):
    """SMART learning goal"""
    id: str = Field(..., description="Unique goal identifier")
    category: LearningCategory
    original_input: str = Field(..., description="User's original goal input")
    smart_version: str = Field(..., description="SMART formatted goal")
    specialty: str
    setting: str
    created_at: datetime
    completed_at: Optional[datetime] = None

    class Config:
        json_encoders = {
            datetime: lambda v: v.isoformat()
        }

class CategoryProgress(BaseModel):
    """Progress tracking for a category"""
    category: LearningCategory
    completed_goals: List[SmartGoal] = Field(default_factory=list)
    total_goals: int = Field(default=0)


In [ ]:
#| export
class LearningContext:
    """
    Manages learning context and goal tracking.
    
    This class handles:
    - Rotation context management
    - SMART goal tracking
    - Progress monitoring by category
    - Context persistence
    """
    
    def __init__(self, context_path: Optional[Path] = None):
        """
        Initialize learning context.
        
        Args:
            context_path: Optional path to load/save context
        """
        self.context_path = context_path
        
        # Initialize rotation context
        self.rotation = RotationContext(
            specialty="",
            setting=""
        )
        
        # Initialize category tracking
        self.category_progress: Dict[LearningCategory, CategoryProgress] = {
            cat: CategoryProgress(category=cat)
            for cat in LearningCategory
        }
        
        # Current active goal
        self.active_goal: Optional[SmartGoal] = None
        
        # Load existing context if available
        if context_path and context_path.exists():
            self.load_context()
    
    def update_rotation(self, specialty: str, setting: str) -> None:
        """
        Update rotation context.
        
        Args:
            specialty: Medical specialty
            setting: Clinical setting
        """
        self.rotation = RotationContext(
            specialty=specialty,
            setting=setting
        )
        self._save_context()
    
    def add_smart_goal(self, goal: SmartGoal) -> None:
        """
        Add new SMART goal.
        
        Args:
            goal: SMART goal to add
        """
        self.active_goal = goal
        self.category_progress[goal.category].total_goals += 1
        self._save_context()
    
    def complete_active_goal(self) -> None:
        """Complete current active goal."""
        if self.active_goal:
            self.active_goal.completed_at = datetime.now()
            cat = self.active_goal.category
            self.category_progress[cat].completed_goals.append(self.active_goal)
            self.active_goal = None
            self._save_context()
    
    def get_category_summary(self) -> Dict[str, Dict]:
        """
        Get summary of progress by category.
        
        Returns:
            dict: Category summaries including completed/total goals
                 and recent completions
        """
        return {
            cat.value: {
                "completed": len(prog.completed_goals),
                "total": prog.total_goals,
                "recent": [
                    {
                        "smart_version": goal.smart_version,
                        "completed_at": goal.completed_at.isoformat()
                    }
                    for goal in sorted(
                        prog.completed_goals,
                        key=lambda x: x.completed_at or datetime.min,
                        reverse=True
                    )[:3]  # Last 3 completed
                ]
            }
            for cat, prog in self.category_progress.items()
        }
    
    def get_all_goals(self) -> List[Dict]:
        """
        Get all goals for current rotation.
        
        Returns:
            list: All goals matching current rotation
        """
        all_goals = []
        for prog in self.category_progress.values():
            # Add completed goals
            all_goals.extend([
                goal.dict() for goal in prog.completed_goals
                if (goal.specialty == self.rotation.specialty and
                    goal.setting == self.rotation.setting)
            ])
            
            # Add active goal if matches
            if self.active_goal and (
                self.active_goal.specialty == self.rotation.specialty and
                self.active_goal.setting == self.rotation.setting
            ):
                all_goals.append(self.active_goal.dict())
        
        return sorted(
            all_goals,
            key=lambda x: x["created_at"],
            reverse=True
        )
    
    def load_context(self) -> None:
        """Load context from file."""
        try:
            with open(self.context_path, 'r') as f:
                data = json.load(f)
            
            # Load rotation
            self.rotation = RotationContext(**data["rotation"])
            
            # Load progress
            for cat_data in data["category_progress"]:
                cat = LearningCategory(cat_data["category"])
                
                # Convert goal dicts back to models
                completed_goals = [
                    SmartGoal(**g) for g in cat_data["completed_goals"]
                ]
                
                self.category_progress[cat] = CategoryProgress(
                    category=cat,
                    completed_goals=completed_goals,
                    total_goals=cat_data["total_goals"]
                )
            
            # Load active goal if exists
            if data.get("active_goal"):
                self.active_goal = SmartGoal(**data["active_goal"])
                
            logger.info(f"Context loaded from {self.context_path}")
            
        except Exception as e:
            logger.error(f"Error loading context: {str(e)}")
    
    def _save_context(self) -> None:
        """Save context to file."""
        if not self.context_path:
            return
            
        try:
            data = {
                "rotation": self.rotation.dict(),
                "category_progress": [
                    {
                        "category": prog.category,
                        "completed_goals": [
                            g.dict() for g in prog.completed_goals
                        ],
                        "total_goals": prog.total_goals
                    }
                    for prog in self.category_progress.values()
                ],
                "active_goal": self.active_goal.dict() if self.active_goal else None
            }
            
            with open(self.context_path, 'w') as f:
                json.dump(data, f, indent=2)
                
            logger.info(f"Context saved to {self.context_path}")
            
        except Exception as e:
            logger.error(f"Error saving context: {str(e)}")

## Tests

In [ ]:
#| hide
def test_learning_context():
    """Test learning context functionality"""
    # Initialize context
    context = LearningContext()
    
    try:
        # Test rotation updates
        context.update_rotation("Emergency Medicine", "ED")
        assert context.rotation.specialty == "Emergency Medicine"
        assert context.rotation.setting == "ED"
        
        # Test learning objectives
        goal = SmartGoal(
            id="test_goal_1",
            category=LearningCategory.CLINICAL_REASONING,
            original_input="Get better at chest pain",
            smart_version="Demonstrate systematic assessment of chest pain",
            specialty="Emergency Medicine",
            setting="ED",
            created_at=datetime.now()
        )
        
        context.add_smart_goal(goal)
        assert context.active_goal == goal
        
        # Test completing objectives
        context.complete_active_goal()
        assert context.active_goal is None
        
        # Test progress tracking
        summary = context.get_category_summary()
        cat_data = summary[LearningCategory.CLINICAL_REASONING.value]
        assert cat_data["completed"] == 1
        assert cat_data["total"] == 1
        assert len(cat_data["recent"]) == 1
        
        # Test getting all goals
        all_goals = context.get_all_goals()
        assert len(all_goals) == 1
        assert all_goals[0]["id"] == "test_goal_1"
        
    finally:
        # Cleanup if needed
        pass
        
    print("All learning context tests passed!")

# Run tests
if __name__ == "__main__":
    test_learning_context()

2025-01-18 23:31:55,450 - __main__ - INFO - Learning context initialized
2025-01-18 23:31:55,457 - __main__ - INFO - Learning context initialized


All learning context tests passed!
